# Denoising with the deep decoder

The code below demonstrates the denoising performance on an example image.

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
#%matplotlib notebook

import glob
import heapq
import os
import pickle
from pprint import pprint
import random
import time

import warnings
warnings.filterwarnings('ignore')

from include import *
from PIL import Image
import PIL

import numpy as np
from scipy import stats
import torch
import torch.optim
from torch.autograd import Variable

# Make experiments reproducible with deterministic seed.
torch.manual_seed(1)

GPU = True
if GPU == True:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(1)
    torch.cuda.empty_cache()
    dtype = torch.cuda.FloatTensor
    print("num GPUs",torch.cuda.device_count())
else:
    dtype = torch.FloatTensor


num GPUs 1


### Benchmark

Load the list of benchmark images from the paper:
  
  ```
  Jun Xu, Hui Li, Zhetong Liang, David Zhang, and Lei Zhang
  Real-world Noisy Image Denoising: A New Benchmark
  https://arxiv.org/abs/1804.02603, 2018.
  ```

In [2]:
def benchmark_images(benchmark_dir = "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages/"):
    clean_suffix = '_mean.JPG'
    noisy_suffix = '_real.JPG'
    benchmark_clean_images = sorted(glob.glob(benchmark_dir + '*' + clean_suffix))
    benchmark_noisy_images = sorted(glob.glob(benchmark_dir + '*' + noisy_suffix))
    benchmark_pairs = []
    for clean_image_path in benchmark_clean_images:
        root_filename = clean_image_path[:-len(clean_suffix)]
        noisy_image_path = root_filename + noisy_suffix
        assert noisy_image_path in benchmark_noisy_images
        benchmark_pairs.append({"clean": clean_image_path, "noisy": noisy_image_path})
    return benchmark_pairs

In [3]:
BENCHMARK_IMAGES = benchmark_images()
print("Loaded %d benchmark pairs." % len(BENCHMARK_IMAGES))
pprint("Example: %s" % ["%d: %s" % (k,v['clean']) for k,v in enumerate(BENCHMARK_IMAGES)])

Loaded 100 benchmark pairs.
("Example: ['0: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_3200_chair_11_mean.JPG', "
 "'1: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_3200_chair_14_mean.JPG', "
 "'2: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_3200_chair_5_mean.JPG', "
 "'3: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_3200_plug_11_mean.JPG', "
 "'4: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_3200_plug_12_mean.JPG', "
 "'5: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_6400_bicycle_10_mean.JPG', "
 "'6: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\Canon5D2_5_160_6400_bicycle_6_mean.JPG', "
 "'7: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages

 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5-6_160_6400_wall_15_mean.JPG', "
 "'64: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5-6_160_6400_wall_2_mean.JPG', "
 "'65: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5-6_160_6400_wall_6_mean.JPG', "
 "'66: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5-6_160_6400_wall_9_mean.JPG', "
 "'67: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5_100_4000_flower_12_mean.JPG', "
 "'68: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5_100_4000_flower_13_mean.JPG', "
 "'69: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5_100_4000_flower_14_mean.JPG', "
 "'70: "
 "./test_data/PolyU-Real-World-Noisy-Images-Dataset-master/CroppedImages\\\\NikonD800_5_100_4000_flo

In [18]:
def benchmark_statistics():
    snrs = []
    ssims = []
    durations = []
    for benchmark_number in range(len(BENCHMARK_IMAGES)):
        existing_picklefiles = glob.glob("benchmark_results/%d_*_benchmark_result.p" % benchmark_number)
        if len(existing_picklefiles) > 0:
            existing_picklefile = existing_picklefiles[0]
            snr, ssim, duration, img_clean_np, img_noisy_np, img_out_np = pickle.load(open(existing_picklefile, "rb"))
            snrs.append(float(snr))
            ssims.append(float(ssim))
            durations.append(float(duration))
            print("%d, %s, %.3f, %.3f, %.3f" % (benchmark_number, os.path.basename(BENCHMARK_IMAGES[benchmark_number]['clean']), snr, ssim, duration))
    print(stats.describe(snrs))
    print(stats.describe(ssims))
    print(stats.describe(durations))

In [19]:
benchmark_statistics()

0, Canon5D2_5_160_3200_chair_11_mean.JPG, 39.533, 0.978, 1259.801
1, Canon5D2_5_160_3200_chair_14_mean.JPG, 37.802, 0.938, 1256.939
2, Canon5D2_5_160_3200_chair_5_mean.JPG, 40.091, 0.970, 1254.908
3, Canon5D2_5_160_3200_plug_11_mean.JPG, 36.428, 0.970, 1259.123
4, Canon5D2_5_160_3200_plug_12_mean.JPG, 35.602, 0.959, 1260.493
5, Canon5D2_5_160_6400_bicycle_10_mean.JPG, 33.592, 0.918, 1260.297
6, Canon5D2_5_160_6400_bicycle_6_mean.JPG, 33.969, 0.913, 1259.963
7, Canon5D2_5_160_6400_bicycle_8_mean.JPG, 33.842, 0.960, 1225.049
8, Canon5D2_5_160_6400_circuit_11_mean.JPG, 32.123, 0.931, 1210.743
9, Canon5D2_5_160_6400_circuit_3_mean.JPG, 33.414, 0.943, 1217.092
10, Canon5D2_5_160_6400_circuit_8_mean.JPG, 33.907, 0.946, 1224.778
11, Canon5D2_5_160_6400_desk_10_mean.JPG, 35.342, 0.977, 1232.849
12, Canon5D2_5_160_6400_desk_4_mean.JPG, 35.107, 0.968, 1222.722
13, Canon5D2_5_160_6400_desk_5_mean.JPG, 33.521, 0.974, 1226.579
14, Canon5D2_5_160_6400_desk_7_mean.JPG, 34.512, 0.960, 1225.208
15, Can